# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [130]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [132]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9666666666666667

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [136]:
import matplotlib.pyplot as plt
import numpy as np
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

In [137]:
# x = np.linspace(0, 2*np.pi, 1000); plt.figure(); y = np.sin(9*x); plt.plot(x, y)
# plt.scatter([0, np.pi, 2*np.pi, np.pi/2, 19*np.pi/18], [0, 0, 0, 1, -1], s=50, c="r", zorder=3)

In [138]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

num_arboles = np.random.randint(5, 50, 5)
nivel_profundidad = np.random.randint(2, 10, 5)

num_arboles, nivel_profundidad

(array([33, 14, 43,  8, 48]), array([2, 2, 6, 4, 8]))

In [139]:
resultados_f1 = []

for i in range(len(nivel_profundidad)):
    modelo, f1 = Bosque.RegresionBosque(X, y, num_arboles[i], nivel_profundidad[i])
    resultados_f1.append(f1)

resultados_f1

[0.9333333333333333,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556]

In [140]:
arboles_v = num_arboles.reshape([-1, 1])
profundidad_v = nivel_profundidad.reshape([-1, 1])

X_hiperparametros = np.hstack((arboles_v, profundidad_v))
X_hiperparametros

array([[33,  2],
       [14,  2],
       [43,  6],
       [ 8,  4],
       [48,  8]])

In [141]:
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(X_hiperparametros, resultados_f1)

In [142]:
x_1 = np.linspace(0, 50, 100).reshape([-1, 1])
x_2 = np.linspace(2, 10, 100).reshape([-1, 1])
m_matricial = np.hstack((x_1, x_2))

In [143]:
prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

In [144]:
nueva_X = np.vstack((X_hiperparametros, m_matricial[i_prox]))
nueva_X

array([[33.        ,  2.        ],
       [14.        ,  2.        ],
       [43.        ,  6.        ],
       [ 8.        ,  4.        ],
       [48.        ,  8.        ],
       [31.31313131,  7.01010101]])

In [145]:
padi = m_matricial[i_prox]
padi
#Si estos dos no se ponen como enteros me marca un error
padi_0 = int(padi[0])
padi_1 = int(padi[1])

In [146]:
modelo, f1 = Bosque.RegresionBosque(X, y, (padi_0), (padi_1))
resultados_f1.append(f1)
resultados_f1

[0.9333333333333333,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9444444444444444]

In [147]:
# Segunda iteración 
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(nueva_X, resultados_f1)

prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

padi = m_matricial[i_prox]
nueva_X2 = np.vstack((nueva_X, padi))

modelo, f1 = Bosque.RegresionBosque(X, y, 46, 9) 
resultados_f1.append(f1)


print("Iteración 2:")
print(resultados_f1)

Iteración 2:
[0.9333333333333333, 0.9333333333333333, 0.9555555555555556, 0.9444444444444444, 0.9555555555555556, 0.9444444444444444, 0.9555555555555556]


In [148]:
# Tercera iteración 
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(nueva_X, resultados_f1)

prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

padi = m_matricial[i_prox]
nueva_X3 = np.vstack((nueva_X2, padi))

modelo, f1 = Bosque.RegresionBosque(X, y, 41, 8)  
resultados_f1.append(f1)

print("Iteración 3:")
print(resultados_f1)

ValueError: Found input variables with inconsistent numbers of samples: [6, 7]

In [ ]:
# Cuarta iteración 
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(nueva_X, resultados_f1)

prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

padi = m_matricial[i_prox]
nueva_X4 = np.vstack((nueva_X3, padi))

modelo, f1 = Bosque.RegresionBosque(X, y, 36, 7)  
resultados_f1.append(f1)

print("Iteración 4:")
print(resultados_f1)

In [ ]:
# Quinta iteración 
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(nueva_X, resultados_f1)

prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

padi = m_matricial[i_prox]
nueva_X5 = np.vstack((nueva_X4, padi))

modelo, f1 = Bosque.RegresionBosque(X, y, 31, 6) 
resultados_f1.append(f1)

print("Iteración 5:")
print(resultados_f1)

In [ ]:
# Sexta iteración 
kernel = 1.0 * RBF(length_scale=1)
modelo_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(nueva_X, resultados_f1)

prediccion_y, desviacionstd_y = modelo_gp.predict(m_matricial, return_std=True)
prediccion_y_alto = prediccion_y + 1.96 * desviacionstd_y
i_prox = np.argmax(prediccion_y_alto)

padi = m_matricial[i_prox]
nueva_X5 = np.vstack((nueva_X4, padi))

modelo, f1 = Bosque.RegresionBosque(X, y, 26, 5)  
resultados_f1.append(f1)

print("Iteración 6:")
print(resultados_f1)

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [152]:
import numpy as np
import random

X, y = datasets.load_iris(return_X_y=True)
v_arboles = np.arange(5, 51)  # Valores para árboles
v_profundidad = np.arange(2, 11)  # Valores para profundidad

T = 10
T_min = 0.1
alpha = 0.9

arbol_actual = random.choice(v_arboles)
profundidad_actual = random.choice(v_profundidad)
modelo, f1_actual = Bosque.RegresionBosque(X, y, arbol_actual, profundidad_actual)

mejor_f1 = f1_actual
mejores_hiperparametros = (arbol_actual, profundidad_actual)

while T > T_min:
    if random.choice([True, False]):
        nuevo_arbol = random.choice(v_arboles)
        nueva_profundidad = profundidad_actual
    else:
        nuevo_arbol = arbol_actual
        nueva_profundidad = random.choice(v_profundidad)

    modelo, nuevo_f1 = Bosque.RegresionBosque(X, y, nuevo_arbol, nueva_profundidad)

    p = np.exp((nuevo_f1 - f1_actual) / T)
    if nuevo_f1 > f1_actual or random.random() < p:
        arbol_actual = nuevo_arbol
        profundidad_actual = nueva_profundidad
        f1_actual = nuevo_f1

    if f1_actual > mejor_f1:
        mejor_f1 = f1_actual
        mejores_hiperparametros = (arbol_actual, profundidad_actual)

    T = T * alpha

mejor_f1

0.9666666666666667